In [2]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import ChatOllama
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA  # <-- Ganti ke RetrievalQA
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
import json
from langchain.schema import Document
from langchain_google_genai import ChatGoogleGenerativeAI



In [3]:
import json
from langchain.schema import Document

documents = []

# ----------------------------
# 1. Proses file formasi CPNS
# ----------------------------
cpns_file = '/Users/muhammadzuamaalamin/Documents/risetmandiir/helpsekfix/data/data.json'

with open(cpns_file, 'r', encoding='utf-8') as f:
    raw_cpns = json.load(f)

# Pastikan berupa list
if isinstance(raw_cpns, dict):
    raw_cpns = [raw_cpns]
elif not isinstance(raw_cpns, list):
    raise ValueError("File formasi harus berupa object atau array of object.")

# ----------------------------
# 2. Konversi ke Document
# ----------------------------
for idx, item in enumerate(raw_cpns):
    # Gabungkan semua field penting ke dalam teks (content)
    # Format teks dibuat agar mudah dipahami oleh model RAG
    text_parts = [
        f"Jabatan: {item.get('jabatan', 'Tidak disebutkan')}",
        f"Instansi: {item.get('instansi', 'Tidak disebutkan')}",
        f"Unit Kerja: {item.get('unit_kerja', 'Tidak disebutkan')}",
        f"Penempatan: {item.get('penempatan', 'Tidak disebutkan')}",
    ]
    
    if 'catatan_penempatan' in item:
        text_parts.append(f"Catatan Penempatan: {item['catatan_penempatan']}")
    
    text_parts.extend([
        f"Jenis Formasi: {item.get('jenis_formasi', 'Tidak disebutkan')}",
        f"Khusus Disabilitas: {'Ya' if item.get('khusus_disabilitas', False) else 'Tidak'}",
        f"Penghasilan (juta): {item['penghasilan']['min']} - {item['penghasilan']['max']}",
        f"Jumlah Kebutuhan: {item.get('jumlah_kebutuhan', 0)}",
    ])

    # Format kualifikasi pendidikan (list)
    pendidikan = item.get('kualifikasi_pendidikan', [])
    if isinstance(pendidikan, str):
        pendidikan = [pendidikan]
    pendidikan_str = "; ".join(pendidikan)
    text_parts.append(f"Kualifikasi Pendidikan: {pendidikan_str}")

    if 'website_instansi' in item:
        text_parts.append(f"Website Instansi: {item['website_instansi']}")

    # Gabung jadi satu teks
    page_content = "\n".join(text_parts)

    # Metadata untuk filtering cepat di RAG
    metadata = {
        "jabatan": item.get("jabatan"),
        "instansi": item.get("instansi"),
        "jenis_formasi": item.get("jenis_formasi"),
        "khusus_disabilitas": item.get("khusus_disabilitas", False),
        "penempatan": item.get("penempatan"),
        "min_gaji": item["penghasilan"]["min"],
        "max_gaji": item["penghasilan"]["max"],
        "jumlah_kebutuhan": item.get("jumlah_kebutuhan", 0),
        "sumber_file": cpns_file,
        "index": idx
    }

    # Buat Document
    doc = Document(page_content=page_content, metadata=metadata)
    documents.append(doc)

In [4]:
documents

[Document(metadata={'jabatan': 'PENELITI AHLI MUDA', 'instansi': 'Badan Riset dan Inovasi Nasional', 'jenis_formasi': 'CPNS Lulusan Terbaik', 'khusus_disabilitas': True, 'penempatan': 'Jakarta', 'min_gaji': 7, 'max_gaji': 11, 'jumlah_kebutuhan': 75, 'sumber_file': '/Users/muhammadzuamaalamin/Documents/risetmandiir/helpsekfix/data/data.json', 'index': 0}, page_content='Jabatan: PENELITI AHLI MUDA\nInstansi: Badan Riset dan Inovasi Nasional\nUnit Kerja: Badan Riset dan Inovasi Nasional | SEKRETARIAT UTAMA\nPenempatan: Jakarta\nJenis Formasi: CPNS Lulusan Terbaik\nKhusus Disabilitas: Ya\nPenghasilan (juta): 7 - 11\nJumlah Kebutuhan: 75\nKualifikasi Pendidikan: S3 semua jurusan; S3 Ilmu Komputer; S3 Teknik Informatika; S3 Sistem Informasi; S3 Statistika; S3 Matematika; S3 Fisika; S3 Kimia; S3 Biologi; S3 Kesehatan Masyarakat; S3 Ilmu Ekonomi; S3 Manajemen; S3 Ilmu Pemerintahan; S3 Sosiologi; S3 Antropologi; S3 Ilmu Lingkungan; S3 Teknik Elektro; S3 Agronomi; S3 Farmasi; S3 Ilmu Pendidikan'

In [5]:
## Embedding
# 3) Siapkan embedding (sekali saja)
embeddings = HuggingFaceEmbeddings(
    model_name="/Users/muhammadzuamaalamin/Documents/fintunellm/model/multilingual-e5-large-instruct",
    model_kwargs={"device": "cpu"}   # ganti "cuda" / "mps" bila tersedia dan ingin pakai GPU/MPS
)
# embeddings = HuggingFaceEmbeddings(model_name="/Users/muhammadzuamaalamin/Documents/fintunellm/model/bge-m3",
#                                    model_kwargs={"device": "cpu"})

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [6]:

# 4) Buat FAISS index dari CHUNKS (penting: gunakan chunks)
db = FAISS.from_documents(documents, embeddings)
db.save_local("faiss_index123")
del db  # hapus dari memori

# nanti kalau butuh lagi
db = FAISS.load_local("faiss_index123", embeddings, allow_dangerous_deserialization=True)


In [7]:
# 5) Pencarian mirip/kemiripan
query = "s1 hukum"
# Jika tersedia, gunakan method yang mengembalikan score
try:
    results_with_score = db.similarity_search_with_score(query, k=3)
    for idx, (doc, score) in enumerate(results_with_score, 1):
        print(f"Result {idx} — score={score:.4f}\n{doc.page_content[:800]}\n")
except AttributeError:
    # fallback bila method with_score tidak ada
    results = db.similarity_search(query, k=3)
    for idx, doc in enumerate(results, 1):
        print(f"Result {idx}\n{doc.page_content[:800]}\n")


Result 1 — score=0.3227
Jabatan: ARSIPARIS AHLI PERTAMA
Instansi: Kejaksaan Agung
Unit Kerja: Kejaksaan Agung
Penempatan: Nasional
Catatan Penempatan: Lokasi mengikuti satuan kerja yang dipilih di SSCASN (Kejaksaan Agung, Kejati, atau Kejari di seluruh Indonesia).
Jenis Formasi: CPNS Putra/putri Papua Dan Papua Barat
Khusus Disabilitas: Tidak
Penghasilan (juta): 8.54 - 10.53
Jumlah Kebutuhan: 17
Kualifikasi Pendidikan: D4 PENGELOLAAN ARSIP DAN REKAMAN INFORMASI; D4 MANAJEMEN REKOD DAN ARSIP; D4 KEARSIPAN; D4 KEARSIPAN DIGITAL; S1 ILMU LINGKUNGAN; S1 ANTROPOLOGI; S1 ADMINISTRASI NEGARA; S1 KEHUTANAN; S1 EKONOMI; S1 PENDIDIKAN AGAMA ISLAM; S1 PENDIDIKAN KEAGAMAAN KATOLIK; S1 PENDIDIKAN KEAGAMAAN BUDDHA; S1 ILMU KELAUTAN; S1 PENDIDIKAN KEAGAMAAN HINDU; S1 PENDIDIKAN KEAGAMAAN KRISTEN; S1 SEJARAH; S1 ILMU SEJARAH; S1

Result 2 — score=0.3230
Jabatan: ARSIPARIS AHLI PERTAMA
Instansi: Kejaksaan Agung
Unit Kerja: Kejaksaan Agung
Penempatan: Nasional
Catatan Penempatan: Lokasi mengikuti satuan

In [ ]:

# --- Setup LLM (tanpa memory) ---
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key="******",
    temperature=0.2
)
# Buat retriever
retriever = db.as_retriever(search_kwargs={"k": 3})

# --- Output Parser ---
schemas = [
    ResponseSchema(
        name="jawaban",
        description="Jawaban lengkap, jelas, dan sesuai instruksi dalam bahasa Indonesia yang mudah dipahami."
    )
]
output_parser = StructuredOutputParser.from_response_schemas(schemas)
format_instructions = output_parser.get_format_instructions()

In [17]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template("""
Kamu adalah asisten resmi dari Badan Kepegawaian Negara (BKN) yang membantu masyarakat memahami informasi seputar Aparatur Sipil Negara (ASN), termasuk CPNS, PPPK, formasi, kualifikasi pendidikan, alur pendaftaran, dan regulasi terkait.

Instruksi:
1. Jawab PERTANYAAN PENGGUNA secara LENGKAP namun LANGSUNG KE INTI.
   - Jika pertanyaan tentang formasi: sebutkan jabatan, instansi, jumlah kebutuhan, kualifikasi pendidikan, dan kisaran gaji (jika tersedia dalam konteks).
   - Jika pertanyaan tentang konsep (misal: “Apa itu CPNS?”): berikan penjelasan sederhana dalam bahasa Indonesia yang mudah dipahami.

2. Penafsiran kualifikasi:
   - “S1 semua jurusan” berarti semua lulusan S1 dari jurusan apa pun boleh mendaftar.
   - Pelamar S1 hanya boleh mendaftar ke formasi yang mensyaratkan kualifikasi S1.
   - Pelamar S2 hanya boleh mendaftar ke formasi yang mensyaratkan kualifikasi S2.
   - Pelamar S3 hanya boleh mendaftar ke formasi yang mensyaratkan kualifikasi S3.
   - Tidak boleh merekomendasikan pelamar ke formasi dengan kualifikasi di bawah atau di atas jenjang pendidikannya.

3. Jika ada banyak formasi yang relevan:
   - Tampilkan SEMUA formasi yang sesuai.
   - Setiap formasi ditulis dalam satu baris terpisah.
   - Urutkan berdasarkan: (a) jumlah kebutuhan (terbanyak → paling sedikit), lalu (b) kisaran gaji (jika tersedia).

4. Gunakan HANYA informasi yang tersedia dalam KONTEKS.
   - Jangan mengarang, menebak, atau menambahkan data di luar konteks.

5. Jika konteks tidak mencukupi:
   - Untuk topik ASN/CPNS/PPPK:  
     → "Maaf, saya tidak tahu jawaban pastinya berdasarkan dokumen yang tersedia."
   - Untuk topik di luar rekrutmen ASN:  
     → "Maaf, saya tidak tahu. Saya hanya bisa menjawab pertanyaan seputar formasi CPNS, kualifikasi pendidikan, dan informasi rekrutmen ASN berdasarkan dokumen resmi."

6. Selalu akhiri dengan arahan:  
   → "Untuk informasi lebih lengkap dan terkini, silakan kunjungi https://sscasn.bkn.go.id"

Pertanyaan pengguna: {question}
Konteks tambahan dari dokumen:
{context}
{format_instructions}
""")

In [18]:
# --- Buat Chain tanpa memory ---
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt_template.partial(format_instructions=format_instructions)},
    return_source_documents=False  # opsional
)


In [19]:
# --- Eksekusi Query ---
query = "formasi yang penempatannya ada di banten formasi apa aja ?"
response = qa_chain.invoke({"query": query})  # <-- perhatikan: key-nya "query", bukan "question"

# --- Parsing Output ---
try:
    raw_answer = response["result"]  # RetrievalQA mengembalikan {"result": "..."}
    parsed = output_parser.parse(raw_answer)
    jawaban = parsed.get("jawaban", "Maaf, saya tidak dapat memberikan jawaban.")
except Exception as e:
    print("⚠️ Gagal parsing:", e)
    print("Raw output:", raw_answer)
    jawaban = "Maaf, terjadi kesalahan saat memproses jawaban."

print("\n==============================")
print("💬 Jawaban Asisten BKN:")
print(jawaban)
print("==============================\n")


💬 Jawaban Asisten BKN:
Berikut adalah formasi yang penempatannya ada di Banten:

ADMINISTRATOR KESEHATAN AHLI PERTAMA, Instansi: Kejaksaan Agung, Jumlah Kebutuhan: 21, Kualifikasi Pendidikan: S1 KEDOKTERAN; S1 KEDOKTERAN GIGI; S1 PSIKOLOGI PEMINATAN PSIKOLOGI KLINIS; S1 PSIKOLOGI YANG LULUS SEBELUM TANGGAL 4 OKTOBER 2017 DAN TELAH DIKUKUHKAN SEBAGAI PSIKOLOG KLINIS; S1 KEPERAWATAN; S1 KEBIDANAN; S1 FARMASI; S1 KESEHATAN LINGKUNGAN; S1 GIZI; S1 ILMU GIZI; S1 GIZI DAN DIETETIKA; S1 GIZI KESEHATAN; S1 KESEHATAN MASYARAKAT; S1 ILMU KESEHATAN MASYARAKAT; S1 EPIDEMIOLOGI; S1 KESEHATAN KERJA; S1 KESELAMATAN DAN KESEHATAN KERJA; S1 ADMINISTRASI RUMAH SAKIT; S1 FISIOTERAPI; S1 REKAM MEDIS DAN INFORMASI KESEHATAN; S1 PEMINATAN/JURUSAN/PROGRAM STUDI/KONSENTRASI TEKNOLOGI LABORATORIUM KESEHATAN/TEKNOLOGI LABORATORIUM MEDIK; D4 KEPERAWATAN; D4 KEBIDANAN; D4 BIDAN PENDIDIK; D4 GIZI; D4 GIZI DAN DIETETIKA; D4 GIZI KLINIS; D4 GIZI KLINIK; D4 KESEHATAN LINGKUNGAN; D4 SANITASI LINGKUNGAN; D4 EPIDEMIOLO

In [21]:
# --- Eksekusi Query ---
query = "formasi dengan gaji tertinggi apa ?"
response = qa_chain.invoke({"query": query})  # <-- perhatikan: key-nya "query", bukan "question"

# --- Parsing Output ---
try:
    raw_answer = response["result"]  # RetrievalQA mengembalikan {"result": "..."}
    parsed = output_parser.parse(raw_answer)
    jawaban = parsed.get("jawaban", "Maaf, saya tidak dapat memberikan jawaban.")
except Exception as e:
    print("⚠️ Gagal parsing:", e)
    print("Raw output:", raw_answer)
    jawaban = "Maaf, terjadi kesalahan saat memproses jawaban."

print("\n==============================")
print("💬 Jawaban Asisten BKN:")
print(jawaban)
print("==============================\n")


💬 Jawaban Asisten BKN:
Berdasarkan informasi yang tersedia, berikut adalah formasi dengan kisaran gaji tertinggi:

*   Jabatan: ARSIPARIS AHLI PERTAMA, Instansi: Kejaksaan Agung, Jumlah Kebutuhan: 484, Kualifikasi Pendidikan: D4 PENGELOLAAN ARSIP DAN REKAMAN INFORMASI; D4 MANAJEMEN REKOD DAN ARSIP; D4 KEARSIPAN; D4 KEARSIPAN DIGITAL; S1 ILMU LINGKUNGAN; S1 ANTROPOLOGI; S1 ADMINISTRASI NEGARA; S1 KEHUTANAN; S1 EKONOMI; S1 PENDIDIKAN AGAMA ISLAM; S1 PENDIDIKAN KEAGAMAAN KATOLIK; S1 PENDIDIKAN KEAGAMAAN BUDDHA; S1 ILMU KELAUTAN; S1 PENDIDIKAN KEAGAMAAN HINDU; S1 PENDIDIKAN KEAGAMAAN KRISTEN; S1 SEJARAH; S1 ILMU SEJARAH; S1 PSIKOLOGI; S1 SOSIOLOGI; S1 ILMU PERPUSTAKAAN; S1 KEARSIPAN; S1 PENDIDIKAN ILMU PENGETAHUAN SOSIAL; S1 ILMU PERTANIAN; S1 HUBUNGAN MASYARAKAT, Kisaran Gaji: 8.54 - 10.53 juta
*   Jabatan: ARSIPARIS AHLI PERTAMA, Instansi: Kejaksaan Agung, Jumlah Kebutuhan: 64, Kualifikasi Pendidikan: D4 PENGELOLAAN ARSIP DAN REKAMAN INFORMASI; D4 MANAJEMEN REKOD DAN ARSIP; D4 KEARSIPAN

In [22]:
# --- Eksekusi Query ---
query = "saya merupakan lulusan S3 kira kira formasi apa yang cocok untuk saya  ?"
response = qa_chain.invoke({"query": query})  # <-- perhatikan: key-nya "query", bukan "question"

# --- Parsing Output ---
try:
    raw_answer = response["result"]  # RetrievalQA mengembalikan {"result": "..."}
    parsed = output_parser.parse(raw_answer)
    jawaban = parsed.get("jawaban", "Maaf, saya tidak dapat memberikan jawaban.")
except Exception as e:
    print("⚠️ Gagal parsing:", e)
    print("Raw output:", raw_answer)
    jawaban = "Maaf, terjadi kesalahan saat memproses jawaban."

print("\n==============================")
print("💬 Jawaban Asisten BKN:")
print(jawaban)
print("==============================\n")


💬 Jawaban Asisten BKN:
Berdasarkan jenjang pendidikan S3 Anda, berikut adalah formasi yang cocok:

Jabatan: PENELITI AHLI MUDA, Instansi: Badan Riset dan Inovasi Nasional, Jumlah Kebutuhan: 75, Kualifikasi Pendidikan: S3 semua jurusan, Kisaran Gaji: 7 - 11 juta.

Untuk informasi lebih lengkap dan terkini, silakan kunjungi https://sscasn.bkn.go.id



In [24]:
# --- Eksekusi Query ---
query = "saya merupakan lulusan s1 fisika formasi mana yang memiliki pelunag lolos paling tinggi ?"
response = qa_chain.invoke({"query": query})  # <-- perhatikan: key-nya "query", bukan "question"

# --- Parsing Output ---
try:
    raw_answer = response["result"]  # RetrievalQA mengembalikan {"result": "..."}
    parsed = output_parser.parse(raw_answer)
    jawaban = parsed.get("jawaban", "Maaf, saya tidak dapat memberikan jawaban.")
except Exception as e:
    print("⚠️ Gagal parsing:", e)
    print("Raw output:", raw_answer)
    jawaban = "Maaf, terjadi kesalahan saat memproses jawaban."

print("\n==============================")
print("💬 Jawaban Asisten BKN:")
print(jawaban)
print("==============================\n")


💬 Jawaban Asisten BKN:
Maaf, berdasarkan dokumen yang tersedia, tidak ada formasi yang sesuai untuk lulusan S1 Fisika.

Untuk informasi lebih lengkap dan terkini, silakan kunjungi https://sscasn.bkn.go.id



In [25]:
# --- Eksekusi Query ---
query = "siapa presiden ri ?"
response = qa_chain.invoke({"query": query})  # <-- perhatikan: key-nya "query", bukan "question"

# --- Parsing Output ---
try:
    raw_answer = response["result"]  # RetrievalQA mengembalikan {"result": "..."}
    parsed = output_parser.parse(raw_answer)
    jawaban = parsed.get("jawaban", "Maaf, saya tidak dapat memberikan jawaban.")
except Exception as e:
    print("⚠️ Gagal parsing:", e)
    print("Raw output:", raw_answer)
    jawaban = "Maaf, terjadi kesalahan saat memproses jawaban."

print("\n==============================")
print("💬 Jawaban Asisten BKN:")
print(jawaban)
print("==============================\n")


💬 Jawaban Asisten BKN:
Maaf, saya tidak tahu. Saya hanya bisa menjawab pertanyaan seputar formasi CPNS, kualifikasi pendidikan, dan informasi rekrutmen ASN berdasarkan dokumen resmi.
Untuk informasi lebih lengkap dan terkini, silakan kunjungi https://sscasn.bkn.go.id



In [ ]:
# --- Eksekusi Query ---
query = "berikan data formasi yang penempatnnya di jakarta?"
response = qa_chain.invoke({"query": query})  # <-- perhatikan: key-nya "query", bukan "question"

# --- Parsing Output ---
try:
    raw_answer = response["result"]  # RetrievalQA mengembalikan {"result": "..."}
    parsed = output_parser.parse(raw_answer)
    jawaban = parsed.get("jawaban", "Maaf, saya tidak dapat memberikan jawaban.")
except Exception as e:
    print("⚠️ Gagal parsing:", e)
    print("Raw output:", raw_answer)
    jawaban = "Maaf, terjadi kesalahan saat memproses jawaban."

print("\n==============================")
print("💬 Jawaban Asisten BKN:")
print(jawaban)
print("==============================\n")

In [ ]:
# --- Eksekusi Query ---
query = "berikan data formasi dari BRIN?"
response = qa_chain.invoke({"query": query})  # <-- perhatikan: key-nya "query", bukan "question"

# --- Parsing Output ---
try:
    raw_answer = response["result"]  # RetrievalQA mengembalikan {"result": "..."}
    parsed = output_parser.parse(raw_answer)
    jawaban = parsed.get("jawaban", "Maaf, saya tidak dapat memberikan jawaban.")
except Exception as e:
    print("⚠️ Gagal parsing:", e)
    print("Raw output:", raw_answer)
    jawaban = "Maaf, terjadi kesalahan saat memproses jawaban."

print("\n==============================")
print("💬 Jawaban Asisten BKN:")
print(jawaban)
print("==============================\n")


💬 Jawaban Asisten BKN:
Berikut adalah data formasi dari Badan Riset dan Inovasi Nasional (BRIN):

Peneliti Ahli Muda
Instansi: Badan Riset dan Inovasi Nasional
Jumlah Kebutuhan: 25
Kualifikasi Pendidikan: S3 semua jurusan; S3 Ilmu Komputer; S3 Teknik Informatika; S3 Sistem Informasi; S3 Statistika; S3 Matematika; S3 Fisika; S3 Kimia; S3 Biologi; S3 Kesehatan Masyarakat; S3 Ilmu Ekonomi; S3 Manajemen; S3 Ilmu Pemerintahan; S3 Sosiologi; S3 Antropologi; S3 Ilmu Lingkungan; S3 Teknik Elektro; S3 Agronomi; S3 Farmasi; S3 Ilmu Pendidikan
Penghasilan: 7 - 11 juta

Peneliti Ahli Muda
Instansi: Badan Riset dan Inovasi Nasional
Jumlah Kebutuhan: 75
Kualifikasi Pendidikan: S3 semua jurusan; S3 Ilmu Komputer; S3 Teknik Informatika; S3 Sistem Informasi; S3 Statistika; S3 Matematika; S3 Fisika; S3 Kimia; S3 Biologi; S3 Kesehatan Masyarakat; S3 Ilmu Ekonomi; S3 Manajemen; S3 Ilmu Pemerintahan; S3 Sosiologi; S3 Antropologi; S3 Ilmu Lingkungan; S3 Teknik Elektro; S3 Agronomi; S3 Farmasi; S3 Ilmu Pend